In [1]:
import pandas as pd
import re
from urllib.parse import unquote
from openpyxl import load_workbook
import requests
from tqdm import tqdm

In [2]:
df = pd.read_csv("EMAIL VARIFICATION.csv",encoding='ISO-8859-1')
df.head()

,E-Mail Address,Domain,CA,status,reason
0,sona.madhu@gmail.com,gmail.com,100003866,valid,email passed all checks
1,deepaksahni9966@gmail.com,gmail.com,152459337,valid,email passed all checks
2,jaideepkoli996@gmail.com,gmail.com,101934014,valid,email passed all checks
3,manishaverma@gmail.com,gmail.com,151653767,valid,email passed all checks
4,ashishjhun.india@gmail.com,gmail.com,153133777,valid,email passed all checks


In [3]:
import re
from urllib.parse import unquote

def clean_email(email):
    # Step 1: Basic cleanup
    email = str(email).strip().lower()
    email = re.sub(r'\s+', '', email)              # remove spaces
    email = re.sub(r'[\'\",\\><]+', '', email)     # remove quotes, \, >, <, etc.
    email = unquote(email)                         # URL decode if needed

    # Step 2: Handle multiple '@' symbols
    if email.count('@') > 1:
        parts = email.split('@')
        email = ''.join(parts[:-1]) + '@' + parts[-1]

    # Step 3: Handle "@." or ".@" issues
    if '@' in email:
        user, domain = email.split('@', 1)
        user = user.rstrip('.')         # Remove trailing . from username
        domain = domain.lstrip('.')     # Remove leading . from domain
        email = f"{user}@{domain}"

    # Step 4: Clean domain part after valid TLD
    valid_tlds = ['.com', '.in', '.co.in', '.org', '.net', '.gov', '.edu']
    if '@' in email:
        user, domain = email.split('@', 1)
        for tld in valid_tlds:
            if tld in domain:
                tld_index = domain.find(tld) + len(tld)
                domain = domain[:tld_index]  # trim anything after the TLD
                break
        # Final cleanup of domain if any symbols remain
        domain = re.sub(r'[^\w\.-]', '', domain)
        email = f"{user}@{domain}"

    return email


In [4]:
df['E-Mail Address'] = df['E-Mail Address'].apply(clean_email)

In [5]:
print(df['Domain'].value_counts().to_string())

Domain
gmail.com                         101670
yahoo.com                           4881
yahoo.co.in                         2231
rediffmail.com                      1003
hotmail.com                          929
yahoo.in                             673
gufum.com                            425
iubridge.com                         371
nic.in                               296
ymail.com                            259
outlook.com                          202
gov.in                               190
tormails.com                         170
icloud.com                           129
mail.com                             123
live.com                             116
email.com                             99
eil.co.in                             51
relianceada.com                       49
rocketmail.com                        47
yahoo.co.uk                           45
ignou.ac.in                           33
emlhub.com                            32
emlpro.com                            31
g.com    

In [6]:
df.head()

,E-Mail Address,Domain,CA,status,reason
0,sona.madhu@gmail.com,gmail.com,100003866,valid,email passed all checks
1,deepaksahni9966@gmail.com,gmail.com,152459337,valid,email passed all checks
2,jaideepkoli996@gmail.com,gmail.com,101934014,valid,email passed all checks
3,manishaverma@gmail.com,gmail.com,151653767,valid,email passed all checks
4,ashishjhun.india@gmail.com,gmail.com,153133777,valid,email passed all checks


In [7]:
domain_corrections = {
    "gamil.com": "gmail.com", "gmai.com": "gmail.com", "gmil.com": "gmail.com",
    "gmal.com": "gmail.com", "gmial.com": "gmail.com", "g-mail.com": "gmail.com",
    "gimal.com": "gmail.com", "gmail.con": "gmail.com", "gamail.com": "gmail.com",
    "gmaill.com": "gmail.com", "gmaol.com": "gmail.com", "gmsil.com": "gmail.com",
    "gmail.om": "gmail.com", "gmqil.com": "gmail.com", "gmali.com": "gmail.com",
    "gmaul.com": "gmail.com", "gmail.co.in": "gmail.com", "gmailo.com": "gmail.com",
    "gmail.vom": "gmail.com", "gemail.com": "gmail.com", "gnail.com": "gmail.com",
    "gimail.com": "gmail.com", "gnmail.com": "gmail.com", "gmail.cpm": "gmail.com",
    "gmailc.om": "gmail.com", "gmail.coom": "gmail.com", "gmail.copm": "gmail.com",
    "gmailm.com": "gmail.com", "gmailcom": "gmail.com", "gmailc.com": "gmail.com",
    "gmail.ocm": "gmail.com", "gmailc.om": "gmail.com", "gmail0.com": "gmail.com",
    "gmail.cpom": "gmail.com", "gmail.in": "gmail.com", "gmâil.com": "gmail.com",
    "gmail.zon": "gmail.com", "gmail.xom": "gmail.com", "gmail.pom": "gmail.com",
    "gmail.omc": "gmail.com", "gmailco.": "gmail.com", "gmail.co": "gmail.com",
    "gmqail.com": "gmail.com", "gmeli.com": "gmail.com", "gmaio.com": "gmail.com",
    "gmile.com": "gmail.com", "gmain.com": "gmail.com", "gmanil.com": "gmail.com",
    "gmaip.com": "gmail.com", "gmaim.com": "gmail.com", "gmails.com": "gmail.com",
    "gmal.cm": "gmail.com", "gmila.com": "gmail.com", "gmamil.com": "gmail.com",
    "gemil.com": "gmail.com", "gmall.com": "gmail.com", "gmmail.com": "gmail.com",
    "gmeil.com": "gmail.com", "gaiml.com": "gmail.com", "gamilcom.": "gmail.com",
    "gamil.co": "gmail.com", "gaimal.com": "gmail.com", "gamial.com": "gmail.com",
    "gmauil.com": "gmail.com", "gmil.co": "gmail.com", "gmasil.com": "gmail.com",
    "gamiil.com": "gmail.com", "gamil.cim": "gmail.com", "gailm.com": "gmail.com",
    "gg-mail.com": "gmail.com", "geml.com": "gmail.com", "gemai.com": "gmail.com",
    "gmil.cpm": "gmail.com", "gmia.com": "gmail.com", "gmaqil.com": "gmail.com",
    "gmatl.com": "gmail.com", "gmaul.con": "gmail.com", "gmeal.com": "gmail.com",
    "gmapil.com": "gmail.com", "g-maill.com": "gmail.com", "gmakl.com": "gmail.com",
    "giemal.com": "gmail.com", "gimai.com": "gmail.com", "gfmail.com": "gmail.com",
    "gnmali.com": "gmail.com" ,"gemil.com": "gmail.com","gmall.com": "gmail.com"
    ,"gmmail.com": "gmail.com","gmeil.com": "gmail.com","gmeli.com": "gmail.com",
    "gmile.com": "gmail.com","gaiml.com": "gmail.com","gamilcom.": "gmail.com",
    "gmnail.com": "gmail.com","gamil.co": "gmail.com","gaimal.com": "gmail.com",
    "gamial.com": "gmail.com","gmauil.com": "gmail.com","gmil.co": "gmail.com",
    "gmanil.com": "gmail.com","gmasil.com": "gmail.com","gmqail.com": "gmail.com",
    "gamiil.com": "gmail.com","gamil.cim": "gmail.com","gailm.com": "gmail.com",
    "gg-mail.com": "gmail.com","geml.com": "gmail.com","gemai.com": "gmail.com",
    "gmila.com": "gmail.com","gmil.cpm": "gmail.com","gmia.com": "gmail.com",
    "gmaqil.com": "gmail.com","gmatl.com": "gmail.com","gmaul.con": "gmail.com",
    "gmeal.com": "gmail.com","gmapil.com": "gmail.com","g-maill.com": "gmail.com",
    "gmakl.com": "gmail.com","gmal.cm": "gmail.com","gmamil.com": "gmail.com",
    "giemal.com": "gmail.com","gimai.com": "gmail.com","gfmail.com": "gmail.com",
    "gnmali.com": "gmail.com","gmaail.com": "gmail.com","gmaiil.com": "gmail.com",
    "gmaik.com": "gmail.com","gma.com": "gmail.com","gmaik.com": "gmail.com",
    "gmaail.com": "gmail.com","gmaiil.com": "gmail.com","gmaik.com": "gmail.com","gmaik.com": "gmail.com",
    "gma.com": "gmail.com","gmaik.com": "gmail.com","yahoomail.com": "yahoo.com","yahooo.com": "yahoo.com",
    "yahho.com": "yahoo.com", "yaoo.com": "yahoo.com","yhaoo.com": "yahoo.com","yaoo.co.in": "yahoo.co.in",
    "yahooco.in": "yahoo.co.in", "yahoo.o.in": "yahoo.co.in","yahoo.coin": "yahoo.com",
    "yaho.co.in": "yahoo.co.in", "yaho.com": "yahoo.com","redifmail.com": "rediffmail.com",
    "reddifmail.com": "rediffmail.com","rediffmail.co": "rediffmail.com","redissmail.com": "rediffmail.com",
    "radiffmail.com": "rediffmail.com","redffmail.com": "rediffmail.com","hotmil.com": "hotmail.com",
    "hotgmail.com": "gmail.com","googlemail.com": "gmail.com","g.mail.com": "gmail.com",
    "gami.com": "gmail.com", "ofgmail.com": "gmail.com","xgmail.com": "gmail.com","tgmail.com": "gmail.com",
    "yahho.co.in": "yahoo.co.in","yaaho.com": "yahoo.com","yajoo.com": "yahoo.com","yahoo.co.i": "yahoo.co.in",
    "yahoo.co.on": "yahoo.co.in","yahoo.om": "yahoo.com","yahoogmail.com": "yahoo.com", "yahoomail.co.in": "yahoo.co.in",
    "yahoo.co..in": "yahoo.co.in","yaho.in": "yahoo.co.in","yaahu.com": "yahoo.com", "outook.com": "outlook.com",
    "oulook.com": "outlook.com","outhei.com": "outlook.com","outlookl.com": "outlook.com","utlook.com": "outlook.com",
    "rediiffmail.com": "rediffmail.com","rediffmal.com": "rediffmail.com","rediffmaill.com": "rediffmail.com",
    "rediffmail.con": "rediffmail.com","redif.com": "rediffmail.com","reiffmail.com": "rediffmail.com",
    "reduffmail.com": "rediffmail.com","refiffmail.com": "rediffmail.com","rediffmai.com": "rediffmail.com",
    "raddifmail.com": "rediffmail.com","rdiffmail.com": "rediffmail.com","reddiffmail.com": "rediffmail.com",
    "bt.com": "bsnl.com","bsnl.co.in": "bsnl.com","aircel.co.in": "aircel.com","airliquide.com": "airliquide.co.in", 
    "relianceda.com": "reliance.com", "rediiffmail.com": "rediffmail.com","mgmail.com": "gmail.com", "gmai.ocm": "gmail.com", 
    "gma8l.com": "gmail.com", "hootmail.com": "hotmail.com",
    "fmail.com": "gmail.com", "jmail.com": "gmail.com", "jkmail.com": "gmail.com", "jkbmail.com": "gmail.com",
    "ociun.com": "oci.gov.in", "bt.com": "bsnl.com", "bsnl.co.in": "bsnl.com", "aircel.co.in": "aircel.com",
    "airliquide.com": "airliquide.co.in", "hootmail.com": "hotmail.com", "edu.email.edu": "edu.in", "gm.com": "gmail.com",
    "hot.mail": "hotmail.com", "yahooo.in": "yahoo.com", "yahoooi.com": "yahoo.com", "yohoo.com": "yahoo.com",
    "yhahoo.co.in": "yahoo.com", "yh00.com": "yahoo.com", "ygmail.com": "gmail.com", "com": "gmail.com",
    "yesbank..in": "yesbank.in", "bajajelectronics": "bajajelectronics.in", "eamil.com": "gmail.com", "eil.co": "eil.co.in",
    "esic.nit.in": "esic.nic.in", "dr.com": "protonmail.com", "dma.asoa": "dma.asia", "delhico.com": "delhi.gov.in",
    "executiveaccess.co.": "executiveaccess.co.in", "eragroup.i": "eragroup.in", "ezwrodlwide.com": "ezworldwide.com", "d102yahoo.co.in": "yahoo.co.in",
    "fco.gov": "gov.uk", "axisbank": "axisbank.com", "gamil.": "gmail.com", "gmael.com": "gmail.com",
    "gmai.co": "gmail.com", "gmai.lcom": "gmail.com", "gmai9l.com": "gmail.com", "gmaifl.com": "gmail.com",
    "gmaii.com": "gmail.com", "gmaiil.co": "gmail.com", "gmail.cpm.com": "gmail.com", "gmailcom.": "gmail.com",
    "gmaile.com": "gmail.com", "gmailk.com": "gmail.com", "gl.com": "gmail.com", "fci.com": "fci.gov.in",
    "gma": "gmail.com", "gma9il.com": "gmail.com", "gmae.com": "gmail.com", "gmill.com": "gmail.com",
    "gmaiul.com": "gmail.com", "gotmail.com": "gmail.com", "hatmail.com": "hotmail.com", "hetmail.com": "hotmail.com",
    "heximail.com": "hotmail.com", "hgmail.com": "gmail.com", "holamail.com": "hotmail.com", "hotmailmail.com": "hotmail.com",
    "hotmali.com": "hotmail.com", "hotmaul.com": "hotmail.com", "hotmial.com": "hotmail.com", "hotamail.com": "hotmail.com",
    "hotmai.com": "hotmail.com", "hotmail..com": "hotmail.com", "hotmail.c": "hotmail.com", "hotmail.co..uk": "hotmail.co.uk",
    "hotmail.con": "hotmail.com", "hotmsil.com": "hotmail.com", "hottmail.com": "hotmail.com", "hptmail.com": "hotmail.com",
    "icloud.co": "icloud.com", "iclout.com": "icloud.com", "iclud.com": "icloud.com", "iclovd.com": "icloud.com",
    "idea.adiyabirla.com": "idea.adityabirla.com", "ieloud.com": "icloud.com", "iiflmail.com": "iifl.com", "gopaljeedairy": "gopaljeedairy.com",
    "clound.com": "cloud.com", "jcdepaux.com": "jcdeaux.com", "jgmail.com": "gmail.com", "indiatimes.co": "indiatimes.com",
    "kkm.india.com": "kkmindia.com", "ktexfabices.com": "ktexfabrics.com", "khullarappearls.com": "khullarapparels.com",
    "apollopharmacy.org": "apollopharmacy.in","appolocranes.in": "apollocranes.in", "coca_cola.m": "coca-cola.com",
    "laste.ml": "laste.com", "latestlabels.com": "latestlabel.com"


}

In [8]:
# Step 3: Clean and correct emails
def correct_email(email):
    if isinstance(email, str) and "@" in email:
        parts = email.strip().split("@")
        if len(parts) == 2:
            local = parts[0].strip()
            domain = parts[1].strip().lower()
            domain = domain.replace(" ", "")  # remove internal spaces
            corrected_domain = domain_corrections.get(domain, domain)
            return f"{local}@{corrected_domain}"
    return email

# Step 4: Apply correction
df["E-Mail Address"] = df["E-Mail Address"].apply(correct_email)

In [9]:
# Step 5: Save corrected emails to the same file
df.to_csv("GmailData.csv", index=False)

print("✅ All emails corrected and saved to GmailData.csv")

✅ All emails corrected and saved to GmailData.csv


In [10]:
df["Domain"] = df["E-Mail Address"].str.extract(r"@([\w\.-]+)")
print(df["Domain"].value_counts().to_string())

Domain
gmail.com                         101669
yahoo.com                           4881
yahoo.co.in                         2231
rediffmail.com                      1003
hotmail.com                          929
yahoo.in                             673
gufum.com                            425
iubridge.com                         371
nic.in                               296
ymail.com                            259
outlook.com                          202
gov.in                               190
tormails.com                         170
icloud.com                           129
mail.com                             123
live.com                             116
email.com                             99
eil.co.in                             51
relianceada.com                       49
rocketmail.com                        47
yahoo.co.uk                           45
ignou.ac.in                           33
emlhub.com                            32
emlpro.com                            31
g.com    

In [11]:
def classify_email(email):
    reason = []
    username, domain = None, None

    try:
        username, domain = email.split('@')
    except ValueError:
        return "invalid", "missing @ or badly formatted"

    # Username checks
    if username.startswith('.') or username.startswith('-') or username.startswith('_'):
        return "invalid", "username starts with invalid character"
    if len(username) < 3:
        return "invalid", "username too short"
    if re.fullmatch(r'\d+', username):
        return "invalid", "username is numeric only"
    if re.search(r'[\.\_\-]{2,}', username):
        return "invalid", "username has multiple special chars"

    # Syntax validation
    regex = r'^[a-zA-Z0-9][a-zA-Z0-9._-]*@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    if not re.match(regex, email):
        return "invalid", "invalid email syntax"

    # Domain check
    disposable_domains = {
    "tempmail.com", "10minutemail.com", "guerrillamail.com", "mailinator.com",
    "yopmail.com", "throwawaymail.com", "fakeinbox.com", "getnada.com",
    "maildrop.cc", "trashmail.com", "sharklasers.com", "emailondeck.com",
    "mintemail.com", "dispostable.com", "spamgourmet.com", "10mail.org"
}

    if domain in disposable_domains:
       return "risky", "disposable email domain"

    # Risky patterns
    if re.fullmatch(r'\d{6,}', username):
        return "risky", "numeric pattern in username"
    
    return "valid", "email passed all checks"

In [12]:
results = df.apply(lambda row: classify_email(row["E-Mail Address"]), axis=1)
df[["status", "reason"]] = pd.DataFrame(results.tolist(), index=df.index)

df = df.drop_duplicates(keep="first").reset_index(drop=True)

# Show result
df.head()

,E-Mail Address,Domain,CA,status,reason
0,sona.madhu@gmail.com,gmail.com,100003866,valid,email passed all checks
1,deepaksahni9966@gmail.com,gmail.com,152459337,valid,email passed all checks
2,jaideepkoli996@gmail.com,gmail.com,101934014,valid,email passed all checks
3,manishaverma@gmail.com,gmail.com,151653767,valid,email passed all checks
4,ashishjhun.india@gmail.com,gmail.com,153133777,valid,email passed all checks


In [13]:
df.to_csv("EMAIL VARIFICATION.csv", index=False)
print("Done")

Done


In [14]:
df = pd.read_csv('EMAIL VARIFICATION.csv')
df['status'].value_counts()

status
valid      118267
invalid       456
risky          38
Name: count, dtype: int64

In [15]:
df['reason'].value_counts()

reason
email passed all checks                   118267
username too short                           177
invalid email syntax                         133
username is numeric only                     117
disposable email domain                       38
username starts with invalid character        18
username has multiple special chars           10
missing @ or badly formatted                   1
Name: count, dtype: int64

In [16]:
df['CA'].value_counts().sum()

np.int64(118761)

In [17]:
df.shape

(118761, 5)